In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [2]:
# パスの確認↓
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


①スパムテキストのダウンロード
https://github.com/kujirahand/spam-database-ja

②Livedoorニュースコーパス
https://www.rondhuit.com/download.html#ldcc

→ gzデータを展開するアプリが必要なため、Teamsから「livedoornews.zip」をDLしてください。

③ ①と②のファイルをColab環境にアップロード

④ アップロードした①と②をColab上で展開（unzip）する。

ヒント：効果測定対策練習問題／問2

In [ ]:
!unzip 'spam-sjis.zip'

In [ ]:
!unzip 'livedoornews.zip'

⑤ P237の作業を行う。

展開済みのフォルダ「livedoornews」のフォルダ名を「ok」に変更する

⑥ P237の作業を行う

展開済みの「spam-sjis」フォルダの中から、「ok」フォルダと同数程度のファイルを「spam」という新規フォルダに移動、もしくはコピーする。

ヒント：P177、glob

ファイルを移動
https://note.nkmk.me/python-shutil-move/

spamデータの「00001.txt」はエンコードのエラーが回避できないので削除しておく

# 以下のソースコードは気が向いたら試してみる程度でOK

※ちょっと時間がかかります。

In [7]:
# 全てのテキストを巡回して単語データベースを作成する
import os, glob
import MeCab
import numpy as np
import pickle

# 保存ファイル名
savefile = "./ok-spam.pickle"
# MeCabの準備
tagger = MeCab.Tagger('/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
# 変数の準備
word_dic = {"__id": 0} # 単語辞書
files = [] # 読み込んだ単語データを追加する

# 指定したディレクトリ内のファイル一覧を読む
def read_files(dir, enccd, label):
    # テキストファイルの一覧を得る
    files = glob.glob(dir + '/*.txt')
    for f in files:
        read_file(f,enccd, label)

# ファイルを読む
def read_file(filename, enccd,label):
    words = []
    # ファイルの内容を読む★
    print('filename',filename)
    with open(filename, "rt", encoding=enccd) as f:
        text = f.read()
    files.append({
        "label": label,
        "words": text_to_ids(text)
    })

# テキストを単語IDのリストに変換
def text_to_ids(text):
    # 形態素解析
    word_s = tagger.parse(text)
    words = []
    # 単語を辞書に登録
    for line in word_s.split("\n"):
        
        if line == 'EOS' or line == '': continue
        word = line.split("\t")[0]

        hinsi = line.split("\t")[1].split(',')[0]
        hinsi2 = line.split("\t")[1].split(',')[1]
        org = line.split('\t')[1].split(',')[6]


        # 助詞・助動詞・記号・数字は捨てる
        if not (hinsi in ['名詞', '動詞', '形容詞']): 
            continue
        if hinsi == '名詞' and hinsi2 == '数詞': continue
        # 単語をidに変換
        id = word_to_id(org)

        words.append(id)
    return words

# 単語をidに変換
def word_to_id(word):

    # 単語が辞書に登録されているか？
    if not (word in word_dic):
        # 登録されていないので新たにIDを割り振る
        id = word_dic["__id"]
        word_dic["__id"] += 1
        word_dic[word] = id
    else:
        # 既存の単語IDを返す
        id = word_dic[word]

    return id

# 単語の頻出頻度のデータを作る
def make_freq_data_allfiles():
    y = []
    x = []
    for f in files:
        y.append(f['label'])
        x.append(make_freq_data(f['words']))
    return y, x

def make_freq_data(words):
    # 単語の出現回数を調べる
    cnt = 0
    dat = np.zeros(word_dic["__id"], 'float')
    for w in words:
        dat[w] += 1
        cnt += 1
    # 回数を出現頻度に直す
    dat = dat / cnt
    return dat



In [8]:
read_files("ok","utf-8", 0)
read_files("spam","shift-jis",1)
y, x = make_freq_data_allfiles()
# ファイルにデータを保存
pickle.dump([y, x, word_dic], open(savefile, 'wb'))
print("単語頻出データ作成完了")

単語頻出データ作成完了


In [13]:
import pickle
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# データファイルの読込
data_file = "./ok-spam.pickle"
save_file = "./ok-spam-model.pickle"
data = pickle.load(open(data_file, "rb"))
y = data[0] # ラベル
x = data[1] # 単語の出現頻度


In [15]:

# 100回、学習とテストを繰り返す
count = 100
rate = 0
for i in range(count):
    # データを学習用とテスト用に分割
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    # 学習する
    model = GaussianNB()
    model.fit(x_train, y_train)
    # 評価する
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    # 評価結果が良ければモデルを保存
    if acc > 0.94: pickle.dump(model, open(save_file, "wb"))
    print(acc)
    rate += acc
# 平均値を表示
print("----")
print("average=", rate / count)



ValueError: ignored

In [11]:
import pickle
import MeCab
import numpy as np
from sklearn.naive_bayes import GaussianNB

# テストするテキスト
test_text1 = """
会社から支給されているiPhoneの調子が悪いのです。
修理に出すので、しばらくはアプリのテストができません。
"""
test_text2 = """
億万長者になる方法を教えます。
すぐに以下のアドレスに返信して。
"""
# ファイル名
data_file = "./ok-spam.pickle"
model_file = "./ok-spam-model.pickle"
label_names = ['OK', 'SPAM']
# 単語辞書を読み出す
data = pickle.load(open(data_file, "rb"))
word_dic = data[2]
# MeCabの準備
tagger = MeCab.Tagger()
# 学習済みモデルを読み出す
model = pickle.load(open(model_file, "rb"))

# テキストがスパムかどうか判定する
def check_spam(text):
    # テキストを単語IDのリストに変換し単語の頻出頻度を調べる
    zw = np.zeros(word_dic['__id'])
    count = 0
    s = tagger.parse(text)
    # 単語毎の回数を加算
    for line in s.split("\n"):
        if line == "EOS": break
        org = line.split('\t')[1].split(',')[6]
        if org in word_dic:
            id = word_dic[org]
            zw[id] += 1
            count += 1
    zw = zw / count
    # 予測
    pre = model.predict([zw])[0]
    print("- 結果=", label_names[pre])

FileNotFoundError: ignored

In [12]:
check_spam(test_text1)
check_spam(test_text2)

NameError: ignored